# Machine Learning Classification

## Telecom Customer Churn Prediction Project

*We use logistic regression to predict churn*

**Dataset** : <https://www.kaggle.com/datasets/blastchar/telco-customer-churn>

